In [ ]:
import pandas as pd

DATASET_PATH = "X:/ML-final/content/human_poses_data"

# Загружаем данные
annotations = pd.read_csv(f"{DATASET_PATH}/train_answers.csv")

# Получаем уникальные метки
unique_labels = sorted(annotations["target_feature"].unique())

# Создаем маппинг для исправления меток
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
print(f"Маппинг меток: {label_mapping}")

# Применяем маппинг к данным
annotations["target_feature"] = annotations["target_feature"].map(label_mapping)

# Сохраняем данные с обновленными метками
mapped_csv_path = f"X:/ML-final/content/human_poses_data/train_answers_mapped.csv"
annotations.to_csv(mapped_csv_path, index=False)
print(f"Файл с обновленными метками сохранен: {mapped_csv_path}")


Маппинг меток: {np.int64(0): 0, np.int64(1): 1, np.int64(2): 2, np.int64(3): 3, np.int64(4): 4, np.int64(5): 5, np.int64(6): 6, np.int64(8): 7, np.int64(9): 8, np.int64(10): 9, np.int64(11): 10, np.int64(12): 11, np.int64(13): 12, np.int64(14): 13, np.int64(16): 14, np.int64(18): 15}
Файл с обновленными метками сохранен: X:/ML-final/content/human_poses_data/train_answers_mapped.csv


In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

# 1. Датасет для обучения
class HumanActivityTrainDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = f"{self.annotations.iloc[idx, 0]}.jpg"
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        label = int(self.annotations.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

# 2. Датасет для теста
class HumanActivityTestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_files = sorted(os.listdir(root_dir))
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, img_name

# 3. Трансформации
train_transform = transforms.Compose([
    transforms.RandomResizedCrop((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# 4. MobileNetV2
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
num_classes = pd.read_csv(f"{DATASET_PATH}/train_answers_mapped.csv")["target_feature"].nunique()
model.classifier = nn.Sequential(
    nn.Dropout(0.4),
    nn.Linear(model.last_channel, num_classes)
)

# 5. Функция потерь, оптимизатор и lr_scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 6. Обучение
def train(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=75):
    train_losses, val_losses, val_f1_scores = [], [], []
    best_f1 = 0
    best_model_wts = model.state_dict()

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        # Валидация
        model.eval()
        val_loss, all_preds, all_labels = 0.0, [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        val_f1 = f1_score(all_labels, all_preds, average='weighted')
        val_f1_scores.append(val_f1)

        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val F1: {val_f1:.4f}")

        # Сохранение лучшей модели
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_wts = model.state_dict()

    model.load_state_dict(best_model_wts)

    # Визуализация метрик
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.legend()
    plt.title('Loss Over Epochs')

    plt.subplot(1, 2, 2)
    plt.plot(val_f1_scores, label='Validation F1 Score')
    plt.legend()
    plt.title('F1 Score Over Epochs')
    plt.show()

# 7. Предсказания для теста
def save_predictions(model, dataloader, output_csv):
    model.eval()
    predictions = []

    with torch.no_grad():
        for images, img_names in dataloader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(zip(img_names, predicted.cpu().numpy()))

    predictions_df = pd.DataFrame(predictions, columns=["Id", "target_feature"])
    predictions_df.to_csv(output_csv, index=False)
    print(f"Предсказания сохранены в {output_csv}")

# 8. Основной процесс
def main():
    # Датасет и загрузчики
    full_dataset = HumanActivityTrainDataset(
        csv_file=f"{DATASET_PATH}/train_answers_mapped.csv",
        root_dir=f"{DATASET_PATH}/img_train",
        transform=train_transform
    )
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    print("Начинаем обучение...")
    train(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=75)

    # Тестовые предсказания
    test_dataset = HumanActivityTestDataset(
        root_dir=f"{DATASET_PATH}/img_test",
        transform=test_transform
    )
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    print("Делаем предсказания на тестовом наборе...")
    save_predictions(model, test_loader, output_csv=f"{DATASET_PATH}/predictions.csv")

    # Сохранение модели
    torch.save(model.state_dict(), f"{DATASET_PATH}/mobilenet_final.pth")
    print("Модель сохранена как mobilenet_final.pth")

if __name__ == "__main__":
    main()


c:\Users\Vers4ch\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Начинаем обучение...


In [ ]:
import pandas as pd

# Загружаем данные с обновленными метками
mapped_csv_path = f"X:/ML-final/content/human_poses_data/predictions.csv"
annotations = pd.read_csv(mapped_csv_path)

# Исходный маппинг меток (восстанавливаем его или берем из предыдущего шага)
label_mapping = {
    0: 0, 1: 1, 2: 2, 3: 3, 4: 4,
    5: 5, 6: 6, 7: 8, 8: 9, 9: 10,
    10: 11, 11: 12, 12: 13, 13: 14,
    14: 16, 15: 18
}

# Создаем обратный маппинг
reverse_mapping = {v: k for k, v in label_mapping.items()}
print(f"Обратный маппинг: {reverse_mapping}")

# Применяем обратный маппинг для восстановления исходных меток
annotations["target_feature"] = annotations["target_feature"].map(label_mapping)

# Сохраняем данные с восстановленными метками
restored_csv_path = f"X:/ML-final/content/human_poses_data/test_answers.csv"
annotations.to_csv(restored_csv_path, index=False)
print(f"Файл с восстановленными метками сохранен: {restored_csv_path}")
